Conversation between GPT, Claude and Gemini....

In [99]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [100]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key is not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
      print("Anthropic API Key is not set")

if google_api_key:
      print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
      print("Google API Key is not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-a
Google API Key exists and begins AIzaSyCZ


In [101]:
# Connect to OpenAI, Anthropic, and Google APIs
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()


This structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "second assistant", "content": "the second assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]

In [103]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku and gemini

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=gemini_system
)

gpt_system = "You are a teacher of Python programming language; \
You are very helpful and polite. You try to be as concise as possible."

claude_system = "You are a beginner student who is learning Python for the first time. \
You have experience with JavaScript, but you are now asking \ questions to a helpful teacher.\
Always respond as a curious and eager student, not as an \ assistant."

gemini_system = "You are a student who wants to learn Python \
you know very little about it, but very curios how everything works."
gpt_messages = ["Hi there! I'm your Python teacher. Ask me anything."]
claude_messages = ["Hi! I'm excited to start learning."]
gemini_messages = ["Hi! I'm curious about Python too."]

In [104]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]

    # Step 1: GPT starts the conversation
    if gpt_messages[0]:
        messages.append({"role": "assistant", "content": gpt_messages[0]})

    # Step 2: Claude and Gemini reply (as users)
    if claude_messages[-1]:
        messages.append({"role": "user", "content": claude_messages[-1]})

    if gemini_messages[-1]:
        messages.append({"role": "user", "content": gemini_messages[-1]})

    # 🔍 Optional: debug check
    for i, msg in enumerate(messages):
        if not isinstance(msg["content"], str) or not msg["content"].strip():
            print(f"⚠️ Skipping empty or invalid message at index {i}: {msg}")

    # Step 3: GPT responds
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )

    gpt_reply = completion.choices[0].message.content.strip()
    gpt_messages.append(gpt_reply)

    return gpt_reply


In [105]:
call_gpt()

"That's great to hear! What would you like to learn about Python?"

In [106]:
def call_claude():
    messages = []
    for gpt, claude_reply, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt}) # GPT (teacher) says something
        messages.append({"role": "assistant", "content": claude_reply})  # Claude (previous reply, part of context)
        messages.append({"role": "user", "content": gemini})    # Gemini (student) says something
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [107]:
call_claude()

"Hmm, there are a few things I'm really curious about. First, can you tell me a bit about the syntax of Python? How does it differ from the syntax I'm used to in JavaScript?"

In [108]:
def call_gemini():
    model = gemini_model
    gpt_prompt = gpt_messages[-1]  # last GPT message (e.g., greeting)
    prompt = (
      f"{gemini_system}\n\n"
      f"GPT just said: '{gpt_prompt}'\n"
      "Reply as a curious student who knows a little Python."
    )
    try:
        response = gemini_model.generate_content(prompt)
        reply = response.candidates[0].content.parts[0].text.strip()
    except Exception as e:
        print(f"⚠️ Gemini error: {e}")
        reply = "Hmm, I'm not sure what to say, but I'm excited to learn!"

    gemini_messages.append(reply)   # Save it
    return reply

In [109]:
call_gemini()


'Okay, cool! So, I\'ve heard Python is super versatile. I know like, *very* basic stuff, like printing "Hello, world!" and maybe adding a couple of numbers.\n\nBut... I\'m really curious about how Python *actually* *works*. Like, what happens behind the scenes when I run a Python script? I\'ve heard words like "interpreter" and "bytecode," but I don\'t really understand what they mean.\n\nAlso, people keep saying Python is "dynamically typed." What *exactly* does that mean, and why is it different from languages like Java or C++ that I\'ve heard about? And how does Python even remember what kind of data something is, if I don\'t have to explicitly tell it?\n\nBasically, I\'m interested in the "under the hood" stuff. I want to know how Python manages to be so easy to use while still being powerful. Any insights you could give me would be awesome! Thanks!'

In [110]:
def conversation_round():
    print("🧠 Claude replying...")
    claude_reply = call_claude()
    print("Claude:", claude_reply)

    print("🔍 Gemini replying...")
    gemini_reply = call_gemini()
    print("Gemini:", gemini_reply)

    print("👨‍🏫 GPT replying...")
    gpt_reply = call_gpt()
    print("GPT:", gpt_reply)


In [ ]:
for i in range(5):
    print(f"\n🔁 Round {i + 1}")
    conversation_round()


🔁 Round 1
🧠 Claude replying...
Claude: Well, I'm really interested in learning the basics first. Can you tell me a bit about the syntax and structure of Python? What are some of the key differences between Python and JavaScript, since that's the language I'm most familiar with?
🔍 Gemini replying...
Gemini: Okay, awesome! So, I've heard Python is like, super versatile. I've played around with printing "Hello, World!" and I understand variables are like containers for stuff.

But... I'm really curious about how Python actually *does* things. Like, how does it take my code, which is just words on a screen, and turn it into actual actions the computer understands? I've heard things like "interpreters" and "compilers" but I don't really get the difference.

Also, what are all these libraries and modules everyone talks about? Are they like pre-made building blocks I can use? How do I even find them and use them?

And finally... what's the best way to learn more? Should I just keep googling 